In [4]:
import os
from clickhouse_driver import Client
import logging
from datetime import timedelta

# Настройки логирования
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Настройки подключения к ClickHouse
clickhouse_host = '10.95.19.132'
clickhouse_port = 9000
clickhouse_user = 'default'
# Рекомендуется хранить пароль в переменных окружения для безопасности
clickhouse_password = os.getenv('CLICKHOUSE_PASSWORD', 'quie1ahpoo5Su0wohpaedae8keeph6bi')
database_name = 'default'
flight_hours_table_name = 'flight_hours'
olap_table_name = 'OlapCube_VNV'

# Функция для заполнения поля daily_flight_hours в таблице OLAP
def update_daily_flight_hours(client):
    try:
        # Получение значения daily_flight_hours из таблицы flight_hours
        flight_hours_query = f"""
            SELECT daily_flight_hours
            FROM {database_name}.{flight_hours_table_name}
            LIMIT 1
        """
        daily_flight_hours = client.execute(flight_hours_query)[0][0]
        logging.info(f"Получено значение daily_flight_hours: {daily_flight_hours}")
    except Exception as e:
        logging.error(f"Ошибка при получении daily_flight_hours из flight_hours: {e}")
        raise

    # Обновление поля daily_flight_hours в OLAP кубе для всех записей на всем протяжении дат
    try:
        update_query = f"""
            ALTER TABLE {database_name}.{olap_table_name}
            UPDATE daily_flight_hours = {daily_flight_hours}
            WHERE daily_flight_hours IS NULL
        """
        client.execute(update_query)
        logging.info(f"Поле daily_flight_hours успешно обновлено в {olap_table_name}")
    except Exception as e:
        logging.error(f"Ошибка при обновлении поля daily_flight_hours в {olap_table_name}: {e}")
        raise

# Подключение к ClickHouse с указанием базы данных
try:
    client = Client(
        host=clickhouse_host,
        port=clickhouse_port,
        user=clickhouse_user,
        password=clickhouse_password,
        database=database_name
    )
    logging.info("Подключение к ClickHouse успешно установлено")
except Exception as e:
    logging.error(f"Не удалось подключиться к ClickHouse: {e}")
    raise

# Обновление поля daily_flight_hours в OLAP кубе
try:
    update_daily_flight_hours(client)
    logging.info("Поле daily_flight_hours успешно обновлено в OLAP кубе")
except Exception as e:
    logging.error(f"Произошла ошибка при обновлении daily_flight_hours: {e}", exc_info=True)


2024-11-28 13:29:42,311 - INFO - Подключение к ClickHouse успешно установлено
2024-11-28 13:29:42,319 - INFO - Получено значение daily_flight_hours: 1.58
2024-11-28 13:29:42,326 - INFO - Поле daily_flight_hours успешно обновлено в OlapCube_VNV
2024-11-28 13:29:42,327 - INFO - Поле daily_flight_hours успешно обновлено в OLAP кубе


In [5]:
import os
from clickhouse_driver import Client
import logging

# Настройки логирования
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Настройки подключения к ClickHouse
CLICKHOUSE_HOST = '10.95.19.132'
CLICKHOUSE_PORT = 9000
CLICKHOUSE_USER = 'default'
# Рекомендуется хранить пароль в переменных окружения для безопасности
CLICKHOUSE_PASSWORD = os.getenv('CLICKHOUSE_PASSWORD', 'quie1ahpoo5Su0wohpaedae8keeph6bi')
DATABASE_NAME = 'default'
AGREGAT_TABLE_NAME = 'Agregat'
OLAP_TABLE_NAME = 'OlapCube_VNV'
REPAIR_TIME_COLUMN = 'RepairTime'
REPAIR_TIME_TYPE = 'Nullable(Float64)'  # Измените тип при необходимости

def connect_to_clickhouse():
    """
    Устанавливает соединение с ClickHouse и возвращает объект клиента.
    """
    try:
        client = Client(
            host=CLICKHOUSE_HOST,
            port=CLICKHOUSE_PORT,
            user=CLICKHOUSE_USER,
            password=CLICKHOUSE_PASSWORD,
            database=DATABASE_NAME
        )
        logging.info("Подключение к ClickHouse успешно установлено")
        return client
    except Exception as e:
        logging.error(f"Не удалось подключиться к ClickHouse: {e}")
        raise

def check_and_add_column(client, table_name, column_name, column_type):
    """
    Проверяет наличие столбца в таблице и добавляет его, если отсутствует.
    """
    try:
        query = f"""
            SELECT COUNT(*)
            FROM system.columns
            WHERE database = '{DATABASE_NAME}'
              AND table = '{table_name}'
              AND name = '{column_name}'
        """
        result = client.execute(query)
        column_exists = result[0][0] > 0
        if column_exists:
            logging.info(f"Столбец '{column_name}' уже существует в таблице '{table_name}'")
        else:
            add_column_query = f"""
                ALTER TABLE {DATABASE_NAME}.{table_name}
                ADD COLUMN {column_name} {column_type}
            """
            client.execute(add_column_query)
            logging.info(f"Столбец '{column_name}' успешно добавлен в таблицу '{table_name}'")
    except Exception as e:
        logging.error(f"Ошибка при проверке или добавлении столбца '{column_name}' в таблицу '{table_name}': {e}")
        raise

def get_repair_time(client, agregat_table):
    """
    Получает значение RepairTime из таблицы Agregat.
    """
    try:
        query = f"""
            SELECT RepairTime
            FROM {DATABASE_NAME}.{agregat_table}
            LIMIT 1
        """
        result = client.execute(query)
        if not result:
            raise ValueError(f"Таблица '{agregat_table}' пуста или столбец '{REPAIR_TIME_COLUMN}' отсутствует.")
        repair_time = result[0][0]
        logging.info(f"Получено значение {REPAIR_TIME_COLUMN}: {repair_time}")
        return repair_time
    except Exception as e:
        logging.error(f"Ошибка при получении {REPAIR_TIME_COLUMN} из таблицы '{agregat_table}': {e}")
        raise

def update_repair_time(client, table_name, repair_time):
    """
    Обновляет столбец RepairTime в таблице OlapCube_VNV для записей, где RepairTime равен NULL.
    """
    try:
        # Используем выражение, которое явно читает столбец RepairTime
        update_query = f"""
            ALTER TABLE {DATABASE_NAME}.{table_name}
            UPDATE {REPAIR_TIME_COLUMN} = {repair_time} + (IFNULL({REPAIR_TIME_COLUMN}, 0) * 0)
            WHERE {REPAIR_TIME_COLUMN} IS NULL
        """
        client.execute(update_query)
        logging.info(f"Столбец '{REPAIR_TIME_COLUMN}' успешно обновлён в таблице '{table_name}'")
    except Exception as e:
        logging.error(f"Ошибка при обновлении столбца '{REPAIR_TIME_COLUMN}' в таблице '{table_name}': {e}")
        raise

def get_table_schema(client, table_name):
    """
    Получает и возвращает схему таблицы.
    """
    try:
        query = f"SHOW CREATE TABLE {DATABASE_NAME}.{table_name}"
        schema = client.execute(query)[0][0]
        logging.info(f"Схема таблицы '{table_name}':\n{schema}")
        return schema
    except Exception as e:
        logging.error(f"Ошибка при получении схемы таблицы '{table_name}': {e}")
        raise

def main():
    # Подключение к ClickHouse
    client = connect_to_clickhouse()
    
    # Проверка схемы таблицы OlapCube_VNV
    try:
        get_table_schema(client, OLAP_TABLE_NAME)
    except Exception as e:
        logging.error(f"Произошла ошибка при проверке схемы таблицы '{OLAP_TABLE_NAME}': {e}", exc_info=True)
        return
    
    # Проверка наличия столбца RepairTime и его добавление при необходимости
    try:
        check_and_add_column(client, OLAP_TABLE_NAME, REPAIR_TIME_COLUMN, REPAIR_TIME_TYPE)
    except Exception as e:
        logging.error(f"Произошла ошибка при проверке или добавлении столбца '{REPAIR_TIME_COLUMN}': {e}", exc_info=True)
        return
    
    # Получение значения RepairTime из таблицы Agregat
    try:
        repair_time = get_repair_time(client, AGREGAT_TABLE_NAME)
    except Exception as e:
        logging.error(f"Произошла ошибка при получении значения '{REPAIR_TIME_COLUMN}': {e}", exc_info=True)
        return
    
    # Обновление столбца RepairTime в таблице OlapCube_VNV
    try:
        update_repair_time(client, OLAP_TABLE_NAME, repair_time)
    except Exception as e:
        logging.error(f"Произошла ошибка при обновлении столбца '{REPAIR_TIME_COLUMN}': {e}", exc_info=True)

if __name__ == "__main__":
    main()


2024-11-28 13:29:46,121 - INFO - Подключение к ClickHouse успешно установлено
2024-11-28 13:29:46,128 - INFO - Схема таблицы 'OlapCube_VNV':
CREATE TABLE default.OlapCube_VNV
(
    `serialno` String,
    `Dates` Date,
    `Status_P` Nullable(String),
    `repair_days` Nullable(Float64),
    `sne` Nullable(Float64),
    `ppr` Nullable(Float64),
    `Status` Nullable(String),
    `daily_flight_hours` Nullable(Float64),
    `daily_flight_hours_f` Nullable(Float64),
    `BR` Nullable(Float64),
    `ll` Nullable(Float64),
    `oh` Nullable(Float64),
    `threshold` Nullable(Float64),
    `Effectivity` Nullable(String),
    `mi8t_count` Nullable(Float64),
    `mi17_count` Nullable(Float64),
    `balance_mi8t` Float64,
    `balance_mi17` Float64,
    `balance_total` Float64,
    `stock_mi8t` Float64,
    `stock_mi17` Float64,
    `stock_total` Float64,
    `location` String,
    `ac_typ` String
)
ENGINE = ReplacingMergeTree
ORDER BY (serialno, Dates)
SETTINGS index_granularity = 8192
2024-11-